источник данных - Вакансии всех регионов России из ЕЦП «Работа в России» (https://trudvsem.ru/opendata/api)

### На какие вопросы можно ответить?

1. какое распределение вакансий по регионам? Абсолютные цифры и на душу населения
2. в каких регионах требуется больше специалистов с ученой степенью?
3. какое распределение вакансий по категориям и регионам?
4. средние зарплаты?
5. динамика зарплат по времени?
6. в каких категориях не указывают зарплаты?

и т.д.100500 вопросов можно придумать)

Также можно будет посмотреть исторические данные и попытаться ответить на вопрос, повлияло ли как-то то или иное событие на рынок вакансий.


### Что предлагается сделать?

Написать приложение, которое будет:
1. скачивать периодически данные из https://trudvsem.ru/opendata/api и сохранять в БД
2. визуализировать данные в BI системе (различные срезы, сегменты и по времени)

Инструменты:
1. clickhouse
2. airflow
3. grafana/datalens
4. python/jupyter notebook
5. docker

Загрузим для примера 10к записей

In [1]:
import requests
import pandas as pd


url = 'http://opendata.trudvsem.ru/api/v1/vacancies'
data = requests.get(url)
data = [vac['vacancy'] for vac in data.json()['results']['vacancies']]
data = pd.DataFrame.from_dict(data, orient='columns')
# adding company attributes
data['company_name'] = data['company'].apply(lambda x: x['name'], 1)
data['companycode'] = data['company'].apply(lambda x: x['companycode'], 1)
data['hr-agency'] = data['company'].apply(lambda x: x['hr-agency'], 1)
# adding region name
data['region_name'] = data['region'].apply(lambda x: x['name'], 1)
# adding specialization
data['specialisation'] = data['category'].apply(lambda x: x['specialisation'], 1)
data.head(3)

,id,source,region,company,creation-date,job-name,vac_url,employment,schedule,duty,...,salary_max,salary_min,salary,code_profession,term,company_name,companycode,hr-agency,region_name,specialisation
0,6755f691-a00d-11ee-9e7c-3950de1bc4b3,Вакансия интернет ресурса,"{'region_code': '6100000000000', 'name': 'Рост...",{'companycode': '7226c750-02f1-11eb-8600-bfd13...,2023-12-21,Персональный брокер,https://trudvsem.ru/vacancy/card/7226c750-02f1...,Полная занятость,Полный рабочий день,<p>БКС Мир инвестиций – международная инвестиц...,...,0,0,NaN,NaN,NaN,Компания БКС,7226c750-02f1-11eb-8600-bfd13399602c,False,Ростовская область,"Продажи, закупки, снабжение, торговля"
1,c5b4aed5-5c3c-11ee-a343-a9b9135e650a,Работодатель,"{'region_code': '4300000000000', 'name': 'Киро...",{'companycode': '5b36bee0-b665-11ed-974a-91f5d...,2023-09-26,Маляр,https://trudvsem.ru/vacancy/card/5b36bee0-b665...,Полная занятость,Полный рабочий день,"<p>Подготовка поверхностей к окрашиванию,&nbsp...",...,0,22129,от 22129,134506,NaN,Филиал ФГБУ 48 Центральный научно-исследовател...,5b36bee0-b665-11ed-974a-91f5d381286a,False,Кировская область,"Строительство, ремонт, стройматериалы, недвижи..."
2,619d2385-8aa1-11ee-8322-e7d0d2cf29b1,Работодатель,"{'region_code': '3100000000000', 'name': 'Белг...","{'companycode': '1023101678500', 'email': 'mdo...",2023-11-24,Преподаватель шахматного кружка на платной основе,https://trudvsem.ru/vacancy/card/1023101678500...,Частичная занятость,Гибкий график,<p>Проведение с воспитанниками ДОУ кружка по ш...,...,19242,19242,от 19242,258179,NaN,МБДОУ Д/С № 64,1023101678500,False,Белгородская область,"Образование, наука"


In [2]:
data.columns

Index(['id', 'source', 'region', 'company', 'creation-date', 'job-name',
       'vac_url', 'employment', 'schedule', 'duty', 'category', 'requirement',
       'addresses', 'social_protected', 'contact_list', 'contact_person',
       'work_places', 'currency', 'salary_max', 'salary_min', 'salary',
       'code_profession', 'term', 'company_name', 'companycode', 'hr-agency',
       'region_name', 'specialisation'],
      dtype='object')

In [3]:
data['specialisation'].value_counts()

Здравоохранение и социальное обеспечение               41
Производство                                           14
Образование, наука                                      9
Транспорт, автобизнес, логистика, склад, ВЭД            9
Продажи, закупки, снабжение, торговля                   7
Туризм, гостиницы, рестораны                            5
Строительство, ремонт, стройматериалы, недвижимость     4
Рабочие специальности                                   3
Информационные технологии, телекоммуникации, связь      2
Искусство, культура и развлечения                       2
Работы, не требующие квалификации                       1
Государственная служба, некоммерческие организации      1
Пищевая промышленность                                  1
Безопасность, службы охраны                             1
Name: specialisation, dtype: int64

In [4]:
data['employment'].value_counts()

Полная занятость       87
Частичная занятость    13
Name: employment, dtype: int64

In [5]:
data['company_name'].nunique()

28

In [6]:
# средняя зарплата (сначала почистим поле от букв и преобразуем к int)
data['salary'] = data['salary'].fillna("0").apply(lambda x: int(x.replace("от ", "")), 1)
data.groupby('employment')['salary'].mean()

employment
Полная занятость       33292.574713
Частичная занятость    14759.461538
Name: salary, dtype: float64